In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import json
from tqdm import tqdm
from scipy.signal import correlate, butter, filtfilt
from statsmodels.tsa.stattools import grangercausalitytests
from joblib import Parallel, delayed
from itertools import product
from pathlib import Path
import warnings
from extract_channels import *

In [ ]:
# Specify the directory containing the tuningTasks files, and 
# the directory to save granger causality results.
data_dir = '/path/to/data/'
gc_session_results_dir = data_dir + 'granger_whole_session/'
results_path = Path(gc_session_results_dir)
results_path.mkdir(parents=True, exist_ok=True)
fiftyWordDat = sio.loadmat(data_dir+'t12.2022.05.03_fiftyWordSet.mat')
sentenceData = sio.loadmat(data_dir+'t12.2022.04.28_sentences.mat')

In [3]:
def get_all_pval(granger_causality_result):
    pval_list = []
    for lag in range(1,len(granger_causality_result)+1):
        pval = granger_causality_result[lag][0]['ssr_ftest'][1]
        pval_list.append(pval)
    return pval_list

In [5]:
def wholefile_gc_tests(data, causal_chans, caused_chans, num_lags, n_jobs=-16):
    spikePow = data['spikePow']
    caused = extract_channel_data(spikePow, caused_chans)
    causal = extract_channel_data(spikePow, causal_chans)
    cue_res = np.zeros((caused.shape[1], causal.shape[1], num_lags))
    
    def compute_gc(chan0, chan1):
        cmp = np.stack([caused[:, chan0], causal[:, chan1]], axis=1)
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=FutureWarning)  # Suppress UserWarnings
            gc_res = grangercausalitytests(cmp, maxlag=num_lags, verbose=False)
        return chan0, chan1, get_all_pval(gc_res)
    
    results = Parallel(n_jobs=n_jobs)(
        delayed(compute_gc)(chan0, chan1)
        for chan0, chan1 in tqdm(product(range(caused.shape[1]), range(causal.shape[1])), total=caused.shape[1] * causal.shape[1])
    )
    
    for chan0, chan1, result in results:
        cue_res[chan0, chan1] = result
    
    return cue_res
    

In [ ]:
def save_wholefile_results(results_np, save_dir, results_filename):
    npy_path = os.path.join(save_dir, results_filename)
    np.save(npy_path, results_np)

In [ ]:
# Sentence-Level Granger Causality Tests
sent_sup44_gc_sup6v_results = wholefile_gc_tests(sentenceData, area_44_superior, area_6v_superior, num_lags=8)
save_wholefile_results(sent_sup44_gc_sup6v_results, gc_session_results_dir, results_filename='sent_sup44_gc_sup6v_results.npy')

sent_sup44_gc_inf6v_results = wholefile_gc_tests(sentenceData, area_44_superior, area_6v_inferior, num_lags=8)
save_wholefile_results(sent_sup44_gc_inf6v_results, gc_session_results_dir, results_filename='sent_sup44_gc_inf6v_results.npy')

sent_inf44_gc_sup6v_results = wholefile_gc_tests(sentenceData, area_44_inferior, area_6v_superior, num_lags=8)
save_wholefile_results(sent_inf44_gc_sup6v_results, gc_session_results_dir, results_filename='sent_inf44_gc_sup6v_results.npy')

sent_inf44_gc_inf6v_results = wholefile_gc_tests(sentenceData, area_44_inferior, area_6v_inferior, num_lags=8)
save_wholefile_results(sent_inf44_gc_inf6v_results, gc_session_results_dir, results_filename='sent_inf44_gc_inf6v_results.npy')

sent_sup6v_gc_sup44_results = wholefile_gc_tests(sentenceData, area_6v_superior, area_44_superior, num_lags=8)
save_wholefile_results(sent_sup6v_gc_sup44_results, gc_session_results_dir, results_filename='sent_sup6v_gc_sup44_results.npy')

sent_sup6v_gc_inf44_results = wholefile_gc_tests(sentenceData, area_6v_superior, area_44_inferior, num_lags=8)
save_wholefile_results(sent_sup6v_gc_inf44_results, gc_session_results_dir, results_filename='sent_sup6v_gc_inf44_results.npy')

sent_inf6v_gc_sup44_results = wholefile_gc_tests(sentenceData, area_6v_inferior, area_44_superior, num_lags=8)
save_wholefile_results(sent_inf6v_gc_sup44_results, gc_session_results_dir, results_filename='sent_inf6v_gc_sup44_results.npy')

sent_inf6v_gc_inf44_results = wholefile_gc_tests(sentenceData, area_6v_inferior, area_44_inferior, num_lags=8)
save_wholefile_results(sent_inf6v_gc_inf44_results, gc_session_results_dir, results_filename='sent_inf6v_gc_inf44_results.npy')


# Word-Level Granger Causality Tests
word_sup44_gc_sup6v_results = wholefile_gc_tests(fiftyWordDat, area_44_superior, area_6v_superior, num_lags=8)
save_wholefile_results(word_sup44_gc_sup6v_results, gc_session_results_dir, results_filename='word_sup44_gc_sup6v_results.npy')

word_sup44_gc_inf6v_results = wholefile_gc_tests(fiftyWordDat, area_44_superior, area_6v_inferior, num_lags=8)
save_wholefile_results(word_sup44_gc_inf6v_results, gc_session_results_dir, results_filename='word_sup44_gc_inf6v_results.npy')

word_inf44_gc_sup6v_results = wholefile_gc_tests(fiftyWordDat, area_44_inferior, area_6v_superior, num_lags=8)
save_wholefile_results(word_inf44_gc_sup6v_results, gc_session_results_dir, results_filename='word_inf44_gc_sup6v_results.npy')

word_inf44_gc_inf6v_results = wholefile_gc_tests(fiftyWordDat, area_44_inferior, area_6v_inferior, num_lags=8)
save_wholefile_results(word_inf44_gc_inf6v_results, gc_session_results_dir, results_filename='word_inf44_gc_inf6v_results.npy')

word_sup6v_gc_sup44_results = wholefile_gc_tests(fiftyWordDat, area_6v_superior, area_44_superior, num_lags=8)
save_wholefile_results(word_sup6v_gc_sup44_results, gc_session_results_dir, results_filename='word_sup6v_gc_sup44_results.npy')

word_sup6v_gc_inf44_results = wholefile_gc_tests(fiftyWordDat, area_6v_superior, area_44_inferior, num_lags=8)
save_wholefile_results(word_sup6v_gc_inf44_results, gc_session_results_dir, results_filename='word_sup6v_gc_inf44_results.npy')

word_inf6v_gc_sup44_results = wholefile_gc_tests(fiftyWordDat, area_6v_inferior, area_44_superior, num_lags=8)
save_wholefile_results(word_inf6v_gc_sup44_results, gc_session_results_dir, results_filename='word_inf6v_gc_sup44_results.npy')

word_inf6v_gc_inf44_results = wholefile_gc_tests(fiftyWordDat, area_6v_inferior, area_44_inferior, num_lags=8)
save_wholefile_results(word_inf6v_gc_inf44_results, gc_session_results_dir, results_filename='word_inf6v_gc_inf44_results.npy')


100%|██████████| 4096/4096 [12:45<00:00,  5.35it/s]
